In [1]:
import requests
import os
import pandas as pd
import acquire

###  Using the code from the lesson as a guide and the REST API from https://python.zach.lol/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [ ]:
response = requests.get('https://python.zach.lol/api/v1/items')
response

In [ ]:
items = response.json()
print(type(items))
items

In [ ]:
base_url = 'https://python.zach.lol'

api_url = base_url + '/api/v1/'
response = requests.get(api_url + 'items')
data = response.json()
    
# create list from 1st page
output = data['payload']['items']

# loop through the pages and add to list
while data['payload']['next_page'] != None:
    
    response = requests.get(base_url + data['payload']['next_page'])
    data = response.json()
    output.extend(data['payload']['items'])
    
df_items = pd.DataFrame(output)

In [ ]:

df_items.head()

###  Do the same thing, but for stores (https://python.zach.lol/api/v1/stores)

In [ ]:
response = requests.get('https://python.zach.lol/api/v1/stores')
response

In [ ]:
stores = response.json()
print(type(stores))
stores

In [ ]:
df_stores = pd.DataFrame(stores['payload']['stores'])
df_stores.head()

### Extract the data for sales (https://python.zach.lol/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [ ]:
response = requests.get('https://python.zach.lol/api/v1/sales')

data = response.json()
data.keys()

In [ ]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

In [ ]:
base_url = 'https://python.zach.lol'

api_url = base_url + '/api/v1/'
response = requests.get(api_url + 'sales')
data = response.json()
    
# create list from 1st page
output = data['payload']['sales']

# loop through the pages and add to list
while data['payload']['next_page'] != None:
    
    response = requests.get(base_url + data['payload']['next_page'])
    data = response.json()
    output.extend(data['payload']['sales'])
    
df_sales = pd.DataFrame(output)

In [ ]:
df_sales

### Save the data in your files to local csv files so that it will be faster to access in the future.

In [ ]:
df_items.to_csv("items_df.csv")
df_stores.to_csv("stores_df.csv")
df_sales.to_csv("sales_df.csv")

### Combine the data from your three separate dataframes into one large dataframe.

In [ ]:
# left join sales and stores
df = pd.merge(df_sales, df_stores, left_on='store', right_on='store_id').drop(columns={'store'})
df.head(2)

In [ ]:
# left join the joined df to the items
df = pd.merge(df, df_items, left_on='item', right_on='item_id').drop(columns={'item'})
df.head(2)

In [ ]:
df.shape

In [ ]:
df.to_csv("joined_df.csv")

### Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [ ]:
url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'

open_power_df = pd.read_csv(url)

In [ ]:
open_power_df.head()

In [ ]:
open_power_df.to_csv("open_power_df.csv")

### Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [ ]:
def get_items_data():
    '''
    This function reads in sales data from a url, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('sales_df.csv'):
        # Reads the csv saved from above, and assigns to the df variable
        df_sales = pd.read_csv('items_df.csv', index_col=0)    
        
    else:
        
        base_url = 'https://python.zach.lol'

        api_url = base_url + '/api/v1/'
        response = requests.get(api_url + 'items')
        data = response.json()
    
        # create list from 1st page
        output = data['payload']['items']

        # loop through the pages and add to list
        while data['payload']['next_page'] != None:
    
            response = requests.get(base_url + data['payload']['next_page'])
            data = response.json()
            output.extend(data['payload']['items'])
    
        df_items = pd.DataFrame(output)
        
    return df_items


In [ ]:
def get_stores_data():
    '''
    This function reads in stores data from a url, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('stores_df.csv'):
        
        # If csv file exists read in data from csv file.
        df_items = pd.read_csv('stores_df.csv', index_col=0)
        
    else:
        
        base_url = 'https://python.zach.lol'
        api_url = base_url + '/api/v1/stores'
        response = requests.get(api_url)
        data = response.json()
        df_items = pd.DataFrame(data['payload']['stores'])
        
        # Cache data
        df.to_csv('zillow_df.csv')
        
    return df_stores

In [ ]:
def get_sales_data():
    '''
    This function reads in sales data from a url, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    
    if os.path.isfile('sales_df.csv'):
        df_sales = pd.read_csv('sales_df.csv', index_col=0)    
        
    else:
        
        base_url = 'https://python.zach.lol'

        api_url = base_url + '/api/v1/'
        response = requests.get(api_url + 'sales')
        data = response.json()
    
        # create list from 1st page
        output = data['payload']['sales']

        # loop through the pages and add to list
        while data['payload']['next_page'] != None:
    
            response = requests.get(base_url + data['payload']['next_page'])
            data = response.json()
            output.extend(data['payload']['sales'])
    
        df_sales = pd.DataFrame(output)
        
    return df_sales


In [ ]:
def get_joined():
    '''
    This function joins the sales, stores and items dataframes into one
    single data frame and return that df.
    '''
    if os.path.isfile('joined_sales_df.csv'):
        joined_sales_df = pd.read_csv('joined_df.csv', index_col=0) 
    else:
        df_items = get_items()
        df_stores = get_stores()
        df_sales = get_sales()
    
        # left join sales and stores
        df = pd.merge(df_sales, df_stores, left_on='store', right_on='store_id').drop(columns={'store'})
    
        # left join the joined df to the items
        df = pd.merge(df, df_items, left_on='item', right_on='item_id').drop(columns={'item'})
    return joined_df

In [ ]:
def get_open_power_data():
    '''
    This function reads in Open Power Systems Data csv file for Germany, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('open_power_df.csv'):
        open_power_df = pd.read_csv('open_power_df.csv', index_col=0)    
        
    else:
        url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
        open_power_df = pd.read_csv(url)
    
        
    return open_power_df


In [3]:
joined_df = acquire.get_joined()

In [4]:
open_power_df = acquire.get_open_power_data()